# alphavec example

This notebook demonstrates running `alphavec.simulate()` on the bundled feather datasets in `tests/testdata/`.

Notes:
- The feather files require `pyarrow`. If you don't have it, install with `pip install pyarrow` in your venv.
- NaNs in prices or funding indicate an asset is not tradable at that period; `simulate()` respects this.
- The run below uses all assets over the full available history.


In [1]:
from pathlib import Path

import numpy as np
import pandas as pd

from alphavec import simulate


In [2]:
data_dir = Path(".")

weights = pd.read_feather(data_dir / "weights.feather")
close_prices = pd.read_feather(data_dir / "close_prices.feather")
order_prices = pd.read_feather(data_dir / "open_prices.feather")
funding_rates = pd.read_feather(data_dir / "funding_rates.feather")

print("weights", weights.shape)
print("close_prices", close_prices.shape)
print("order_prices", order_prices.shape)
print("funding_rates", funding_rates.shape)


weights (1071, 151)
close_prices (1071, 151)
order_prices (1071, 151)
funding_rates (1071, 151)


In [3]:
symbols = weights.columns.tolist()
print("assets", len(symbols))
print("start", weights.index.min())
print("end", weights.index.max())

na_summary = {
    "weights_na": int(weights.isna().sum().sum()),
    "close_na": int(close_prices.isna().sum().sum()),
    "order_na": int(order_prices.isna().sum().sum()),
    "funding_na": int(funding_rates.isna().sum().sum()),
}
na_summary


assets 151
start 2023-01-01 00:00:00+00:00
end 2025-12-06 00:00:00+00:00


{'weights_na': 134921,
 'close_na': 56420,
 'order_na': 56325,
 'funding_na': 67713}

In [4]:
# Use all assets and the full time range.
weights_all = weights.sort_index()

# Align prices/funding to weights; NaNs mean not tradable.
close_all = close_prices.reindex(index=weights_all.index, columns=weights_all.columns)
# Use next-period open as the execution price to avoid lookahead.
open_all = order_prices.reindex(index=weights_all.index, columns=weights_all.columns)
order_all = open_all.shift(-1)
funding_all = funding_rates.reindex(index=weights_all.index, columns=weights_all.columns)

weights_all.head()


symbol,AAVE,ADA,AI,AI16Z,AIXBT,ALT,ANIME,APE,APT,AR,...,ZEN,ZEREBRO,ZK,ZORA,ZRO,kBONK,kFLOKI,kNEIRO,kPEPE,kSHIB
ts,,,,,,,,,,,,,,,,,,,,,
2023-01-01 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-02 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-03 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-04 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-05 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
init_cash = 1_000
benchmark_asset = "BTC"  # set to None to disable alpha/beta

returns, tearsheet = simulate(
    weights=weights_all,
    close_prices=close_all,
    order_prices=order_all,
    funding_rates=funding_all,
    benchmark_asset=benchmark_asset,
    order_notional_min=10,
    fee_pct=0.0004,       # 4 bps per trade
    slippage_pct=0.0002,  # 2 bps per trade
    init_cash=init_cash,
    freq_rule="1D",
    trading_days_year=365,
    risk_free_rate=0.03,
)

returns.head(), tearsheet


(ts
 2023-01-01 00:00:00+00:00    0.0
 2023-01-02 00:00:00+00:00    0.0
 2023-01-03 00:00:00+00:00    0.0
 2023-01-04 00:00:00+00:00    0.0
 2023-01-05 00:00:00+00:00    0.0
 Name: returns, dtype: float64,
 simulation start date       2023-01-01 00:00:00+00:00
 simulation end date         2025-12-06 00:00:00+00:00
 first order date            2023-08-09 00:00:00+00:00
 total return %                             465.932099
 max drawdown (equity) %                    -28.546248
 max drawdown (PnL) %                      -186.671877
 funding earnings                             1.207752
 fees                                       222.980064
 annualized return                            0.805279
 annualized volatility                        0.433575
 annualized sharpe                            1.788108
 annual turnover                             96.553832
 total order count                               42908
 average order notional                      12.991754
 max gross exposure %   

In [6]:
equity = (1.0 + returns).cumprod() * init_cash
equity.tail()


ts
2025-12-02 00:00:00+00:00    5596.495670
2025-12-03 00:00:00+00:00    5574.180093
2025-12-04 00:00:00+00:00    5566.555365
2025-12-05 00:00:00+00:00    5688.505104
2025-12-06 00:00:00+00:00    5659.320986
Name: returns, dtype: float64

## Interactive charts (Plotly)

These charts visualize the portfolio equity, drawdowns, and return distribution.


In [7]:
try:
    import plotly.express as px
    import plotly.graph_objects as go
except ModuleNotFoundError as e:
    raise ModuleNotFoundError(
        "plotly is required for these charts. Install in your venv: pip install plotly"
    ) from e


In [8]:
# Portfolio equity curve
equity_df = equity.reset_index()
equity_df.columns = ["ts", "equity"]

fig_equity = px.line(
    equity_df,
    x="ts",
    y="equity",
    title="Portfolio Equity Curve",
)
fig_equity.update_layout(yaxis_title="Equity")
fig_equity.show()


In [9]:
# Drawdown series
drawdown = equity / equity.cummax() - 1.0
dd_df = drawdown.reset_index()
dd_df.columns = ["ts", "drawdown"]

fig_dd = px.area(
    dd_df,
    x="ts",
    y="drawdown",
    title="Portfolio Drawdown",
)
fig_dd.update_layout(yaxis_title="Drawdown")
fig_dd.update_yaxes(tickformat=".1%")
fig_dd.add_hline(y=0, line_width=1, line_dash="dash", line_color="gray")
fig_dd.show()


In [10]:
# Returns distribution
ret_df = returns.to_frame("returns")

fig_hist = px.histogram(
    ret_df,
    x="returns",
    nbins=100,
    title="Returns Distribution",
    marginal="box",
)
fig_hist.update_layout(xaxis_title="Period Return", yaxis_title="Count")
fig_hist.update_xaxes(tickformat=".2%")
fig_hist.show()
